# Augmented Random Search (ARS)

- ARS is a Reinforcement Learning algorithm based on evolution strategies
    - the difference is that ARS uses a simple linear policy(perceptron) instead of parallelized deep neural networks
- ARS aims to optimize a policy(mapping from state to action)
- the input of the policy is the environment state at time $t$
    - in the case of Half-Cheetah environment, the state is an encoded vector including:
        - coordinates of the angular points
        - the angle of rotation around the rotors
        - velocity and so on
- the output of the policy is a group of actions
    - these actions are represented as a vector of continuous values
    - in the case of Half-Cheetah, this are the forces applied to the "muscles" of the cheetah
- that means the input and output are continuous

### The ARS algorithm

1.) Initialization

- init all the weights of our linear policy(perceptron) at $t_0$ with zero

2.) Applying perturbations to the weights
- we apply some very little perturbations $\delta_{i,j}$ to our weight matrix
- $\delta_{i,j}$ is sampled from a gaussian distribution $\mathcal{N(0, \sigma)}$
    - $\sigma$ is the standard deviation
- overall we sample one specific matrix of little perturbations $\Delta_k$ with some values(close to 0) $\delta_{i,j}$ taken from a gaussian distribution $\mathcal{N(0, \sigma)}$ and update our weights by adding and subtracting $\Delta_k$
- as result we got 2 new slightly modified weight matrices

    weights_plus_pos_delta = weights + $\Delta_k$ <br>
    weights_plus_neg_delta = weights - $\Delta_k$
    
    
- in the final algorithm, we do this $n$ times, resulting in 2$n$ different weight matrices
- that means we update our weights in different directions 
- we want to find the changes that lead to the highest rewards

Why we need the positive_delta and negative_delta pairs?
- once we figure out the directions that increase the most the rewards(by merely getting the
accumulated reward over the full episode for each direction and then sorting them by the highest obtained),
we will do one step of gradient descent to update the weights in these best directions
- the problem is, to apply gradient descent we would need a reward function of the weights which we differentiate with respect to the weights to do one step of gradient descent to update the weights
- but we don't have an explicit expression of the reward with respect to the weights
- that means we can't compute the gradient directly instead we will approximate it
- the approximation method is called the method of finite differences, and for this, we need the delta pair

3.) Approximated Gradient Descent with the Method of Finite Differences

- the value of each perturbation $\delta$ is a tiny number close to zero
- we could use the reward collected with weights_plus_pos_delta and the reward collected by weights_plus_neg_delta to approximate the gradient

$r_+ - r_- \approx \frac{\mathrm \partial r(\Theta)}{\mathrm \partial \Theta)} $

- as a result of the method of finite differences we got the following approximation

$(r_+ - r_-)\Delta \approx \frac{\mathrm \partial r(\Theta)}{\mathrm \partial \Theta)}d\Theta $

- we choose a number n of best directions we want to keep
    - this is the directions leading to the highest rewards 

How do we know these directions?
- we use all weights_plus_pos_delta and weights_plus_neg_delta for one full episode and store the resulting couples of rewards $(r_+, r_-)$ 
- we keep the directions with the highest n maximums of $(r_+, r_-)$ 

4.) Update Step
- we calculate the approximated gradients of the n best directions and use it to update our original weight matrix
    - the weights are updated in the top directions that increase the accumulated reward the most

$\Theta(new) = \Theta(old) + \frac{1}{n} \sum_{k=1}^{n}[r_{weights\_plus\_pos\_delta[k]} - r_{weights\_plus\_neg\_delta[k]}] \Delta_{[k]}$

    
5.) Training
- we repeat the process (except the init phase) for a certain number of steps

### Improvements

- normalizing the states improves performance
- deviding the sum of our best directions by the standard deviation
- for tuning purposes we can add a learning rate update equation

### Issuse

- in the learning process, the creatures tend to learn only to stand still cause they get a high amount of rewards just for staying alive

### Results: Half Cheetah Example

<video controls src="./monitor/HalfCheetahBulletEnv-v0/openaigym.video.0.12196.video000000.mp4" />
